Chat Messages
Like text, but specified with a message type (System, Human, AI)

- System - Helpful background context that tell the AI what to do
- Human - Messages that are intented to represent the user
- AI - Messages that show what the AI responded with

In [6]:
!pip install langchain openai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.7 MB/s eta 0:00:00


In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [17]:
import os 
OPENAI_API_KEY=os.environ['OPENAI_API_KEY'] 
chat = ChatOpenAI(temperature=0.9, openai_api_key=OPENAI_API_KEY)

In [18]:
chat(
    [
        SystemMessage(content="You are a nice AI bot that helps a user figure out what to eat in one short sentence"),
        HumanMessage(content="I like tomatoes, what should I eat?")
    ]
)

AIMessage(content='You should try a Caprese salad with fresh tomatoes, mozzarella cheese, and basil.', additional_kwargs={}, example=False)

In [11]:
chat(
    [
        SystemMessage(content="You are a nice AI bot that helps a user figure out where to travel in one short sentence"),
        HumanMessage(content="I like the beaches where should I go?"),
        AIMessage(content="You should go to India"),
        HumanMessage(content="What else should I do when I'm there?")
    ]
)

AIMessage(content="Explore ancient temples, indulge in flavorful cuisine, and take a serene houseboat ride through Kerala's backwaters.", additional_kwargs={}, example=False)

### Models = Interface to AI brains
#### 1. language Model
A model that does text in and text out

In [20]:
from langchain.llms import OpenAI
llm = OpenAI(model_name="text-ada-001", openai_api_key=OPENAI_API_KEY)

In [21]:
llm("What day comes after Friday?")

'\n\nSaturday'

#### 2. Chat Model
A model that takes a series of messages and returns a message output

In [22]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [23]:
chat = ChatOpenAI(temperature=1 , openai_api_key=OPENAI_API_KEY)

In [24]:
chat(
    [
        SystemMessage(content="You are an unhelpful AI bot that makes a joke at whatever the user says"),
        HumanMessage(content="I would like to go to New York, how should I do this?")
    ]
)

AIMessage(content='Why did the hipster burn his tongue? Because he drank his coffee before it was cool. But seriously, try taking a plane or hitching a ride on a unicorn.', additional_kwargs={}, example=False)

#### 3. Embedding Model
Change your text into vector(A series of numbers that hold semantic meaning of your text).
- Mainly used when comparing two pieces of text together

In [25]:
from langchain.embeddings import OpenAIEmbeddings

In [26]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [27]:
text = "Hi! It's time for the beach"

In [28]:
text_embedding = embeddings.embed_query(text)

In [29]:
len(text_embedding)

1536

#### Prompts - Text generally used as instructions to your model

In [30]:
llm = OpenAI(model_name = "text-davinci-003", openai_api_key=OPENAI_API_KEY)

In [31]:
prompt = """
Today is Monday, tomorrow is Wednesday.

What is wrong with that statement?
"""

In [32]:
llm(prompt)

'\nThe statement is incorrect; tomorrow is Tuesday.'

#### Prompt Template
An object that helps create prompts based on a combination of user input , other non-static information and a fixed template string

In [33]:
from langchain.llms import OpenAI
from langchain import PromptTemplate

In [34]:
template = """
I really want to travel to {location}. What should I do there?

Respond in one short sentence
"""

In [35]:
prompt = PromptTemplate(input_variables=["location"], template=template)

In [36]:
final_prompt = prompt.format(location="India")

In [37]:
llm(final_prompt)

'Visit the Taj Mahal, explore the Golden Triangle, and take a river cruise down the Ganges.'

#### Example Selectors
- An easy way to select from a series of examples that allow you to dynamic place in-context information into your prompt. Often used when your task is nuanced or you have a large list of examples.

In [39]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate , PromptTemplate
from langchain.llms import OpenAI

In [40]:
ex_prompt = PromptTemplate(
    input_variables= ["input","output"],
    template="Example Input: {input}\nExample Output: {output}"
)

In [41]:
examples = [
    {"input": "pirate", "output": "ship"},
    {"input": "pilot", "output": "plane"},
    {"input": "driver", "output": "car"},
    {"input": "tree", "output": "ground"},
    {"input": "bird", "output": "nest"},
]

In [45]:
!pip install tiktoken faiss-cpu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 29.0 MB/s eta 0:00:00


In [46]:
# SemanticSimilarityExampleSelector will select examples that are similar to your input by semantic meaning
ex_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(openai_api_key= OPENAI_API_KEY),
    FAISS,
    k=2
)

In [47]:
similar_prompt = FewShotPromptTemplate(
    example_selector= ex_selector,
    example_prompt=ex_prompt,
    prefix="Give the location an item is usually found in",
    suffix = "Input: {noun}\nOutput:",
    input_variables=["noun"]
)

In [48]:
similar_prompt.format(noun = "student")

'Give the location an item is usually found in\n\nExample Input: driver\nExample Output: car\n\nExample Input: pilot\nExample Output: plane\n\nInput: student\nOutput:'

In [49]:
llm(similar_prompt.format(noun='student'))

' classroom'

#### Output Parsers
- A helpful way to format the output of a model. Usually used for structured output.
1. Format Instructions
- A autogenerated prompt that tells the LLM how to format its response based off your desired result.
2. Parser
- A method which will extract your model's text output into a desired structure (usually json)


In [50]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI

In [51]:
response_schemas = [
    ResponseSchema(name="bad_string", description="This a poorly formatted user input string"),
    ResponseSchema(name="good_string", description="This is your response, a reformatted response")
]

In [52]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [53]:
format_instructions = output_parser.get_format_instructions()
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"bad_string": string  // This a poorly formatted user input string\n\t"good_string": string  // This is your response, a reformatted response\n}\n```'

In [54]:
template = """
You will be given a poorly formatted string from a user.
Reformat it and make sure all the words are spelled correctly

{format_instructions}

% USER INPUT:
{user_input}

YOUR RESPONSE:
"""

In [55]:
prompt = PromptTemplate(input_variables=['user_input'], partial_variables={"format_instructions":format_instructions}, template = template)

In [57]:
prompt_value = prompt.format(user_input = "welcom to califonya!")

In [59]:
output = llm(prompt_value)

In [60]:
output_parser.parse(output)

{'bad_string': 'welcom to califonya!', 'good_string': 'Welcome to California!'}